In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# import os
import random


from empirical_data import *
from optimized_random_response import *
from randomized_response import *
from repetitive_optimizer import *
from synthetic_dataset import *
from exponential_mechanism import *
from simpleinfotheory import *
from normalize_error_matrix import *
from rappor_mechanism import *

import matplotlib as mpl
import matplotlib.lines as mlines

mpl.rcParams.update({'font.size': 14})

ModuleNotFoundError: No module named 'cvxpy'

In [1]:
import cvxpy

ModuleNotFoundError: No module named 'cvxpy'

## Generate Synthetic Datasets

In [ ]:
from alphabet import *

NUM_ATTRIBUTES = 2
NUM_STATES = [3, 3]
DATASET_SIZE = 10000

priority_dict = {} # {'0': 2, '1': 0.5}

TOTAL_STATES = 1
alphabet_dict = {}
for i in range(NUM_ATTRIBUTES):
    TOTAL_STATES *= NUM_STATES[i]
    alphabet_dict[str(i)] = np.arange(NUM_STATES[i])

ALL_ALPHABET = create_alphabet(attributes_with_alphabet=alphabet_dict)
STRING_ALL_ALPHABET = convert_alphabet_to_string(ALL_ALPHABET)
ATTRIBUTE_LIST = list(alphabet_dict.keys())
# random_dist = np.array([random.randint(1, 5000) for i in range(TOTAL_STATES)])
# random_dist = random_dist/np.sum(random_dist)
random_dist = [0.3, 0.01, 0.01, 0.01, 0.30, 0.01, 0.01, 0.01, 0.34] # [0.2, 0.11, 0.01, 0.01, 0.20, 0.11, 0.01, 0.11, 0.24] # [0.3, 0.01, 0.01, 0.01, 0.30, 0.01, 0.01, 0.01, 0.34]
# random_dist = [0.25, 0.25, 0.25, 0.25] # [0.5, 0.025, 0.025, 0.45]

print("Alphabet ", ALL_ALPHABET)
print("ATTRIBUTE_LIST ", ATTRIBUTE_LIST)

synthetic_dataset_constructor = Gen_Synthetic_Dataset(no_of_states = TOTAL_STATES, no_of_samples = DATASET_SIZE, alphabet=STRING_ALL_ALPHABET)
correlated_synthetic_dataset = synthetic_dataset_constructor.gen_custom(distribution=random_dist)

uniform_synthetic_dataset = synthetic_dataset_constructor.gen_uniform()



## Claculate Error Matrix

In [ ]:
# alphabet_dict = {}
# for i in range(len(ALL_ALPHABET)):
#     alphabet_dict[str(ALL_ALPHABET[i])] = i
# print(alphabet_dict)
# normalize_error_matrix = Normalize_error_matrix(attribute_list=ATTRIBUTE_LIST, alphabet=ALL_ALPHABET, priority_dict=priority_dict, alphabet_dict=alphabet_dict)
# err_matrix = normalize_error_matrix.normalized_error_matrix
# sns.heatmap(err_matrix)
# plt.show()

## Calculate Mutual Information

### Pair-wise information leakage of A: I(A;A')

In [ ]:
def mutual_info_pair(a, b):
    return mutualinformationempirical(xn=a, yn=b)[0]

### Total information leakage of A: I(A;A',B')

In [ ]:
# I(A;A',B') = I(A;A') + I(A;B'|A')

def conditional_mutual_info(a, b, c):
    return conditionalmutualinformationempirical(a, b, c)

def total_info_leakage(A, A_prime, B_prime):
    return  mutual_info_pair(A, A_prime) + conditional_mutual_info(A, B_prime, A_prime)
    # return  mutual_info_pair(A, B_prime) - conditional_mutual_info(A, B_prime, A_prime)

In [ ]:
def mutual_information(a, b):
    processed_a = []
    # print(a)
    # print(b)
    for i in a:
        actual_split = i.split(" ")
        actual = []
        # print(actual_split)
        for j in actual_split:
            # print(i)
            if j != "" :
                actual.append(int(j))
        processed_a.append(np.array(actual))
    processed_a = np.array(processed_a)
    processed_b = []
    
    for i in b:
        actual_split = i.split(" ")
        actual = []
        # print(actual_split)
        for j in actual_split:
            # print(i)
            if j != "" :
                actual.append(int(j))
        processed_b.append(np.array(actual))
    processed_b = np.array(processed_b)

    mi = []
    # print("processed_a", processed_a)
    # print("processed_b", processed_b)
    # for i in range(np.shape(processed_b)[1]):
    #     mi.append(mutual_info_pair(processed_a[:,i], processed_b[:,i]))
    # mi.append(mutual_info_pair(processed_a[:,0], processed_a[:,1]))
    # mi.append(mutual_info_pair(processed_b[:,1], processed_b[:,0]))
    # mi.append(mutual_info_pair(processed_a[:,0], processed_b[:,0]))
    # mi.append(mutual_info_pair(processed_a[:,1], processed_b[:,1]))
    # mi.append(total_info_leakage(processed_a[:,0], processed_b[:,0], processed_b[:,1]))
    mi.append(total_info_leakage(processed_a[:,1], processed_b[:,1], processed_b[:,0]))
    # mi.append(total_info_leakage(processed_a[:,0], processed_b[:,1], processed_b[:,0]))
    return mi
    

## Benchmark Optimal Mechanism

In [ ]:
EPS_ARRAY = np.arange(100)*0.15

### Correlated Distribution

In [ ]:
utility_error_function = ["0_1", "l1", "l2"]
line_styles = {"RAPPOR": ":", "k-RR": "--", "Exponential": "-.", "Our": "-"}
    
for util_err in utility_error_function:
    alphabet_dict = {}
    for i in range(len(ALL_ALPHABET)):
        alphabet_dict[str(ALL_ALPHABET[i])] = i
    print(alphabet_dict)
    normalize_error_matrix = Normalize_error_matrix(attribute_list=ATTRIBUTE_LIST, alphabet=ALL_ALPHABET, priority_dict=priority_dict, alphabet_dict=alphabet_dict, err_type=util_err)
    err_matrix = normalize_error_matrix.normalized_error_matrix
    sns.heatmap(err_matrix)
    plt.show()

    exponential_mechanism_without_prior = Exponential_mechanism(prior_dist=random_dist, STATE_COUNT=TOTAL_STATES, INPUT_ALPHABET=STRING_ALL_ALPHABET, normalized_objective_err_matrix=err_matrix, only_err_matrix=True)

    exponential_mechanism_with_prior = Exponential_mechanism(prior_dist=random_dist, STATE_COUNT=TOTAL_STATES, INPUT_ALPHABET=STRING_ALL_ALPHABET, normalized_objective_err_matrix=err_matrix, only_err_matrix=False)

    random_response_mechanism = Randomized_Response(STATE_COUNT=TOTAL_STATES, INPUT_ALPHABET=STRING_ALL_ALPHABET, normalized_objective_err_matrix=err_matrix)

    optimal_random_response_mechanism = Optimized_Randomized_Response(prior_dist = random_dist, STATE_COUNT = TOTAL_STATES, INPUT_ALPHABET = STRING_ALL_ALPHABET, normalized_objective_err_matrix = err_matrix, 
                    TOLERANCE_MARGIN = 0.01, APPROXIMATION = "LINEAR", solver = "SCS", is_kl_div = True, ALPHA=0.01, accelerate_from_rr=True)

    rappor_mechanism = Rappor_mechanism(STATE_COUNT=TOTAL_STATES, INPUT_ALPHABET = STRING_ALL_ALPHABET, prob_f=0.5, prob_p=0.25, prob_q=0.75, collection_count = 20)
    # mechanisms_dict = {"Exp. mechanism without prior": exponential_mechanism_without_prior, "Exp. mechanism with prior": exponential_mechanism_with_prior, "k-RR": random_response_mechanism, "Optimal k-RR": optimal_random_response_mechanism}
    # mechanisms_dict = {"k-RR": random_response_mechanism, "Optimal k-RR": optimal_random_response_mechanism}
    mechanisms_dict = {"RAPPOR": rappor_mechanism, "k-RR": random_response_mechanism, "Exponential": exponential_mechanism_without_prior, "Our": optimal_random_response_mechanism}

    error_dict = {}
    mi_dict = {}
    total_mi_dict = {}

    for key in list(mechanisms_dict.keys()):
        error_dict[key] = []
        mi_dict[key] = []
        total_mi_dict[key] = []

    for eps in EPS_ARRAY:
        # print(eps)
        for mechanism in list(mechanisms_dict.keys()):
            __tot_error = 0
            __perturbed_value_list = []
            for entry in correlated_synthetic_dataset:
                # entry = entry_np_value[0]
                if mechanism == "RAPPOR":
                    __perturbed_value_list.append(mechanisms_dict[mechanism].gen_random_output(actual_value=entry, eps=eps, prob_f=eps/10, is_eps=True)[0])
                else:
                    __perturbed_value_list.append(mechanisms_dict[mechanism].gen_random_output(actual_value=entry, eps=eps)[0])
                # print(entry,__perturbed_value_list[-1])
                __error = normalize_error_matrix.get_value_error(actual=entry, perturbed=__perturbed_value_list[-1])
                __tot_error += __error
            error_dict[mechanism].append(__tot_error/len(uniform_synthetic_dataset))
            mi_dict[mechanism].append(mutual_info_pair(correlated_synthetic_dataset, __perturbed_value_list))
            # total_mi_dict[mechanism].append(total_info_leakage(correlated_synthetic_dataset, __perturbed_value_list, ))
            # mi_dict[mechanism].append(mutual_information(correlated_synthetic_dataset, __perturbed_value_list))

            # if mechanism == "Optimal k-RR":
            #     optimal_mechanism = mechanisms_dict[mechanism].get_mechanism(eps=eps)
                
                # posterior_prob_distribution = np.zeros(TOTAL_STATES)
                # posterior_joint_prob = np.zeros((TOTAL_STATES, TOTAL_STATES))
                # for i in range(TOTAL_STATES):
                #     posterior_joint_prob[:,i] = np.reshape(optimal_mechanism[:,i] * np.array(random_dist), (4,))
                #     posterior_prob_distribution[i] = np.sum(optimal_mechanism[:,i] * np.array(random_dist))
                # print("posterior_prob_distribution ", posterior_prob_distribution)
                # print("Distance ", np.linalg.norm((random_dist-posterior_prob_distribution), 2))
                # sns.heatmap(optimal_mechanism)
                # plt.show()
    lagends = []
    for mechanism in list(mechanisms_dict.keys()):
        lagends.append(mechanism + " utility_error")
        plt.plot(EPS_ARRAY, error_dict[mechanism])
        # lagends.append(mechanism + " info_leakage")
        # plt.plot(EPS_ARRAY, mi_dict[mechanism])
        # lagends.append(mechanism + " total_info_leakage")
        # plt.plot(EPS_ARRAY, total_mi_dict[mechanism])
        # for mi_ in range(len(mi_dict[mechanism][0])):
        plt.plot(EPS_ARRAY, np.array(mi_dict[mechanism]))
        #     lagends.append(f"{mechanism}")
            # legend.append(f"MI for Optimal, attr:{grouped_attributes_list[i][mi_]}")

    plt.legend(lagends)
    plt.xlabel("Privacy Budget (Epsilon)")
    plt.ylabel("Error (Normalized)")
    # plt.title(f"{error_cal_type} error between original value and perturbed value for {Attr_list}")
    plt.show()

    utility_error = np.arange(26)
    
    utility_error = utility_error/np.max(utility_error)

    tolerance = (utility_error[1] - utility_error[0])/2

    info_dict = {}

    for mechanism in list(mechanisms_dict.keys()):
        info_dict[mechanism] = []
        for i in utility_error:
            info_dict[mechanism].append(np.nan)

    for index_1, i in enumerate(utility_error):
        for mechanism in list(mechanisms_dict.keys()):
            for index_, j in enumerate(error_dict[mechanism]):
                if abs(i-j) < tolerance:
                    info_dict[mechanism][index_1] = (mi_dict[mechanism][index_])
                    break
    fig = plt.figure()
    lagends = []
    for mechanism in list(mechanisms_dict.keys()):
        lagends.append(mechanism)
        plt.plot(utility_error, info_dict[mechanism], linestyle=line_styles[mechanism])

    plt.legend(lagends)
    plt.grid(visible=True)
    plt.show()
    fig.savefig(f"{util_err}.pdf", bbox_inches='tight')


In [ ]:
utility_error = np.arange(30)
    
utility_error = utility_error/np.max(utility_error)

tolerance = (utility_error[1] - utility_error[0])/2

info_dict = {}

for mechanism in list(mechanisms_dict.keys()):
    info_dict[mechanism] = []
    for i in utility_error:
        info_dict[mechanism].append(np.nan)

for index_1, i in enumerate(utility_error):
    for mechanism in list(mechanisms_dict.keys()):
        for index_, j in enumerate(error_dict[mechanism]):
            if abs(i-j) < tolerance:
                info_dict[mechanism][index_1] = (mi_dict[mechanism][index_])
                break
fig = plt.figure()
lagends = []
for mechanism in list(mechanisms_dict.keys()):
    lagends.append(mechanism)
    plt.plot(utility_error, info_dict[mechanism], linestyle=line_styles[mechanism])

plt.legend(lagends)
plt.grid(visible=True)
plt.show()
fig.savefig(f"{util_err}.pdf", bbox_inches='tight')


In [ ]:
for mechanism in list(mechanisms_dict.keys()):
    lagends.append(mechanism + " utility_error")
    plt.plot(EPS_ARRAY, error_dict[mechanism])
    # lagends.append(mechanism + " info_leakage")
    # plt.plot(EPS_ARRAY, mi_dict[mechanism])
    # lagends.append(mechanism + " total_info_leakage")
    # plt.plot(EPS_ARRAY, total_mi_dict[mechanism])
    plt.plot(EPS_ARRAY, np.array(mi_dict[mechanism]))
    #     lagends.append(f"{mechanism}")
        # legend.append(f"MI for Optimal, attr:{grouped_attributes_list[i][mi_]}")

plt.legend(lagends)
plt.xlabel("Privacy Budget (Epsilon)")
plt.ylabel("Error (Normalized)")
# plt.title(f"{error_cal_type} error between original value and perturbed value for {Attr_list}")
plt.show()

In [ ]:
error_dict

In [ ]:
utility_error = np.arange(20)
utility_error = utility_error/np.max(utility_error)

tolerance = (utility_error[1] - utility_error[0])/2

info_dict = {}

for mechanism in list(mechanisms_dict.keys()):
    info_dict[mechanism] = []
    for i in utility_error:
        info_dict[mechanism].append(np.nan)

for index_1, i in enumerate(utility_error):
    for mechanism in list(mechanisms_dict.keys()):
        for index_, j in enumerate(error_dict[mechanism]):
            if abs(i-j) < tolerance:
                info_dict[mechanism][index_1] = (mi_dict[mechanism][index_])
                break

lagends = []
for mechanism in list(mechanisms_dict.keys()):
    lagends.append(mechanism + " utility_error")
    plt.plot(utility_error, info_dict[mechanism])

plt.legend(lagends)
plt.grid(visible=True)
plt.show()


In [ ]:
info_dict

In [ ]:
for mechanism in list(mechanisms_dict.keys()):
    lagends.append(mechanism + " utility_error")
    plt.plot(utility_error, info_dict[mechanism])
plt.plot(utility_error, aa)
plt.legend(lagends)
plt.grid(visible=True)
plt.show()

In [ ]:
# aa = info_dict["Optimal k-RR"]

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Sample data with some points set to np.nan
x = np.array([0, 1, 2, 3, 4, 5])
y = np.array([2, 3, np.nan, 5, np.nan, 6])

# Plotting
plt.plot(x, y)
plt.title('Plot with Skipped Points')
plt.xlabel('X-axis')
plt.ylabel('Y-axis')
plt.show()
